In [203]:
from collections import defaultdict, namedtuple, MutableMapping
import pymongo
from multiprocessing import Process,Pipe
from __main__ import *
import json
import treetojson
import pickle
import os
import base64
import csv
import re
import pandas as pd
import numpy as np
import glob
from urllib.parse import urlparse, urlsplit
import requests

In [350]:
def parse(path="C:/Users/aniru/Downloads/fraudulentDataset/dhruval"):
    full_data=[]
    tid=0
    all_files = glob.glob(os.path.join(os.getcwd(), path, "*.txt"))
    for file in all_files:
        data=[]
        with open(file, 'r', encoding='utf8') as f:
            json_dict = json.load(f)
            
            data+=[json_dict['readability']['contentType'],json_dict['language'],json_dict['readability']['charset'],json_dict['subject']]
            
            if json_dict['readability']['contentType'] == "multipart/alternative" : 
                data+=[json_dict['emailLayoutHtml']]
            else:
                data+=[json_dict['emailLayoutText']]

            if json_dict['embeddedURLs']:
                for key in json_dict['embeddedURLs']:
                    tokens =urlparse(key['url'])
                    ####what to do with params=''????      
                    data+=[tokens.netloc,tokens.path,tokens.params]
            
            if json_dict['attachments']:
                data+=[json_dict['attachments'][0]['fileName']]
            tid+=1

            data+=[tid]
            data = [x for x in data if x]
            data = [x for x in data if x != '/']
            
        full_data+=[data]
#         with open("C:/Users/aniru/Desktop/out.csv", "w", newline="") as f:
#             writer = csv.writer(f)
#             writer.writerows(full_data)
#     print(full_data)


    return full_data

In [351]:
a=parse(path="C:/Users/aniru/Downloads/fraudulentDataset/dhruval")
# print(a)


# New Section

In [433]:
def find_frequent_itemsets(transactions, minimum_support, params, include_support=True):
    """
    Find frequent itemsets in the given transactions using FP-growth. This
    function returns a generator instead of an eagerly-populated list of items.
    The `transactions` parameter can be any iterable of iterables of items.
    `minimum_support` should be an integer specifying the minimum number of
    occurrences of an itemset for it to be accepted.
    Each item must be hashable (i.e., it must be valid as a member of a
    dictionary or a set).
    If `include_support` is true, yield (itemset, support) pairs instead of
    just the itemsets.
    """
    items = defaultdict(lambda: 0)  # mapping from items to their supports

    # if using support rate instead of support count
    if 0 < minimum_support <= 1:
        minimum_support = minimum_support * len(transactions)

    # Load the passed-in transactions and count the support that individual
    # items have.
    for transaction in transactions:
        for item in transaction[1:-1]:
            items[item] += 1
    

    
    # items = dict(
    #     (item, support) for item, support in items.items() if support >= minimum_support
    # )
    # Remove infrequent items from the item support dictionary and
    #sorts the dictionary  according to value
    items={k: v for k, v in sorted(items.items(), key=lambda item: item[1],reverse=True) if v>=minimum_support}
    with open('C:/Users/aniru/Desktop/data.json', 'w') as fp:
        json.dump(items, fp)
#     print(items)

    # Build our FP-tree. Before any transactions can be added to the tree, they
    # must be stripped of infrequent items and their surviving items must be
    # sorted in decreasing order of frequency.
    def clean_transaction(transaction):
        
        # transaction[1:] = filter(lambda v: v in items, transaction[1:-1]).append(tid)
        # t=[i for i in items.]
        # transaction[1:-1] = sorted(transaction[1:-1], key=lambda v: items[v], reverse=True)
        tid,temp=transaction[-1],transaction[0]
        transaction=[temp]+[k for k,v in items.items() if k in transaction]+[tid]
     
#         print(transaction)
        return transaction

    master = FPTree()
    for transaction in list(map(clean_transaction, transactions)):
        master.add(transaction)
#     json_str = json.dumps(master)    
#     print(treetojson.get_json(data=master))
        
#     print(master.inspect())
#     find_frequent_itemsets.send(master)
#     find_frequent_itemsets.close()
    return master.getcampaign(params)


   



In [434]:
# class PythonObjectEncoder(json.JSONEncoder):
#     def default(self, obj):
#         return {'_python_object': 
#                 base64.b64encode(pickle.dumps(obj)).decode('utf-8') }

# def as_python_object(dct):
#     if '_python_object' in dct:
#         return pickle.loads(base64.b64decode(dct['_python_object']))
#     return dct

In [435]:
class FPTree(object):
    """
    An FP tree.
    This object may only store transaction items that are hashable
    (i.e., all items must be valid as dictionary keys or set members).
    """

    Route = namedtuple("Route", "head tail")

    def __init__(self):
        # The root node of the tree.
        self._root = FPNode(self, None, None)

        # A dictionary mapping items to the head and tail of a path of
        # "neighbors" that will hit every node containing that item.
        self._routes = {}

    @property
    def root(self):
        """The root node of the tree."""
        return self._root
    

    def add(self, transaction):
        """Add a transaction to the tree."""
        point = self._root

        for item in transaction:
            next_point = point.search(item)
            if next_point:
                # There is already a node in this tree for the current
                # transaction item; reuse it.
                next_point.increment()
            else:
                # Create a new point and add it as a child of the point we're
                # currently looking at.
                next_point = FPNode(self, item)
                point.add(next_point)

                # Update the route of nodes that contain this item to include
                # our new node.
                self._update_route(next_point)

            point = next_point

    def _update_route(self, point):
        """Add the given node to the route through all nodes for its item."""
        assert self is point.tree

        try:
            route = self._routes[point.item]
            route[1].neighbor = point  # route[1] is the tail
            self._routes[point.item] = self.Route(route[0], point)
        except KeyError:
            # First node for this item; start a new route.
            self._routes[point.item] = self.Route(point, point)

    def items(self):
        """
        Generate one 2-tuples for each item represented in the tree. The first
        element of the tuple is the item itself, and the second element is a
        generator that will yield the nodes in the tree that belong to the item.
        """
        for item in self._routes.keys():
            yield (item, self.nodes(item))

    def nodes(self, item):
        """
        Generate the sequence of nodes that contain the given item.
        """

        try:
            node = self._routes[item][0]
        except KeyError:
            return

        while node:
            yield node
    #             print(node)
            node = node.neighbor
    #         print(node)

    def prefix_paths(self, item):
        """Generate the prefix paths that end with the given item."""

        def collect_path(node):
            path = []
            while node and not node.root:
                path.append(node)
                node = node.parent
            path.reverse()
            return path

        return (collect_path(node) for node in self.nodes(item))

    def inspect(self):
        print("Tree:")
        self.root.inspect(1)

        print()
        print("Routes:")
        for item, nodes in self.items():
            print("  %r" % item)
            for node in nodes:
                print("    %r" % node)

    def getcampaign(self,param):
        campaign=[]
        self.root.dfs(param,campaign)
        itemlist=[]
        for item in campaign:
            t=[]
            item.get_items(t)
            itemlist+=[t]
        return itemlist

In [436]:
class FPNode(object):
    """A node in an FP tree."""

    def __init__(self, tree, item, count=1):
        self._tree = tree
        self._item = item
        self._count = count
        self._parent = None
        self._children = {}
        self._neighbor = None
        self._num_leaves=None
        self._items_subtree = None

    def add(self, child):
        """Add the given FPNode `child` as a child of this node."""

        if not isinstance(child, FPNode):
            raise TypeError("Can only add other FPNodes as children")

        if child.item not in self._children:
            self._children[child.item] = child
            child.parent = self

    def remove(self, child):
        """Remove the given FPNode `child` as a child of this node."""

        if not isinstance(child, FPNode):
            raise TypeError("Can only remove other FPNodes as children")
        if child.item in self._children:
            del self._children[child.item]
            
    def disp(self, ind=1):
        print('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

    def search(self, item):
        """
        Check whether this node contains a child node for the given item.
        If so, that node is returned; otherwise, `None` is returned.
        """
        try:
            return self._children[item]
        except KeyError:
            return None

    def __contains__(self, item):
        return item in self._children

    @property
    def tree(self):
        """The tree in which this node appears."""
        return self._tree

    @property
    def item(self):
        """The item contained in this node."""
        return self._item

    @property
    def count(self):
        """The count associated with this node's item."""
        return self._count

    def increment(self):
        """Increment the count associated with this node's item."""
        if self._count is None:
            raise ValueError("Root nodes have no associated count.")
        self._count += 1

    @property
    def root(self):
        """True if this node is the root of a tree; false if otherwise."""
        return self._item is None and self._count is None

    @property
    def leaf(self):
        """True if this node is a leaf in the tree; false if otherwise."""
        return len(self._children) ==0

    @property
    def parent(self):
        """The node's parent"""
        return self._parent

    @parent.setter
    def parent(self, value):
        if value is not None and not isinstance(value, FPNode):
            raise TypeError("A node must have an FPNode as a parent.")
        if value and value.tree is not self.tree:
            raise ValueError("Cannot have a parent from another tree.")
        self._parent = value

    @property
    def neighbor(self):
        """
        The node's neighbor; the one with the same value that is "to the right"
        of it in the tree.
        """
        return self._neighbor

    @neighbor.setter
    def neighbor(self, value):
        if value is not None and not isinstance(value, FPNode):
            raise TypeError("A node must have an FPNode as a neighbor.")
        if value and value.tree is not self.tree:
            raise ValueError("Cannot have a neighbor from another tree.")
        self._neighbor = value

    @property
    def children(self):
        """The nodes that are children of this node."""
        return tuple(self._children.values())
    #         print (tuple(self._children.values()))

    @property
    def num_leaves(self):
        if self.leaf: return 1
        elif self._num_leaves is None: 
            self._num_leaves=sum([i.num_leaves for i in self.children])
        return self._num_leaves 
     
#     @property
#     def items_subtree(self):
#         items = []
#         if self.leaf: return 1
#         elif self._items_subtree is None: 
#             self._items_subtree+=([i.item for i in self._tree.items])
                        
#         return self._items_subtree
    

    def cond1(self,min_num_children):
        return len(self._children) > min_num_children

    def cond2(self,freq_threshold):
        count=0
        n=0
        for child in self.children:
            count+=child.count
            n+=1
        return count/n >= freq_threshold

    def cond3(self,n_obf_features):
        temp_node=self
        while(not temp_node.root):
            if temp_node.item not in n_obf_features: return True
            temp_node=temp_node.parent

        return False
    def cond4(self,min_num_messages):
        return self.num_leaves>min_num_messages

    def cond(self,param):
        return self.cond1(param[0]) and self.cond2(param[1]) and self.cond3(param[2]) and self.cond4(param[3])

    def get_items(self,items):
        items.append(self.item)
        
        if self.leaf: return
            
        for child in self.children:
            child.get_items(items)

    def dfs(self,param,campaign):
        if self.leaf : return
        for child in self.children:
            child.dfs(param,campaign)
        
        if self.cond(param):
            campaign.append(self)
            self.parent.remove(self)
#             print(list(campaign[-1]._tree.items()))
            print(campaign[-1], campaign[-1].num_leaves)
#             myclient = pymongo.MongoClient("mongodb://localhost:27017/")
#             mydb = myclient["nodes"]
#             mycol = mydb["campaigns"]
#             x = mycol.insert_one({"name" : campaign[-1].item})
        return        
        

    def inspect(self, depth=0):
        print(("  " * depth) + repr(self))
        for child in self.children:
            child.inspect(depth + 1)

    def __repr__(self):
        if self.root:
            return "<%s (root)>" % type(self).__name__
        return "<%s %r (%r)>" % (type(self).__name__, self.item, self.count)



In [437]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

In [438]:
simpDat = parse("C:/Users/aniru/Downloads/fraudulentDataset/dhruval")
# simpDat = loadSimpDat()

In [450]:
param=[2,5,['p','t','y'],4]
print(f"No of camp. is {len(find_frequent_itemsets(simpDat,2,param))} ")
# aa=find_frequent_itemsets(simpDat,2,param)
# for aaa in aa: print(aaa)

<FPNode 'iso-8859-1' (85)> 85
<FPNode 'iso-8859-1' (62)> 62
No of camp. is 2 
